In [6]:
import cv2

def track_qr_code():
    # Initialize the video capture object
    cap = cv2.VideoCapture(0)

    # Check if the camera opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file")
        return

    # Read until video is completed
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Detect QR Code in the image
            detector = cv2.QRCodeDetector()
            data, bbox, _ = detector.detectAndDecode(frame)

            # If there is a QR Code
            if bbox is not None:
                # Ensure the bounding box points are in the correct format
                bbox = bbox.astype(int)  # Ensuring the coordinates are integer type
                n_lines = len(bbox[0])

                # Draw each line in the bounding box
                for i in range(n_lines):
                    point1 = tuple(bbox[0][i % n_lines])
                    point2 = tuple(bbox[0][(i + 1) % n_lines])
                    cv2.line(frame, point1, point2, color=(255, 0, 0), thickness=2)

                # Display the decoded text
                if data:
                    text_position = (bbox[0][0][0], bbox[0][0][1] - 10)
                    cv2.putText(frame, data, text_position, cv2.FONT_HERSHEY_SIMPLEX,
                                0.5, (0, 255, 0), 2)

            # Display the resulting frame
            cv2.imshow('QR Code Tracker', frame)

            # Press Q on keyboard to exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # Release the video capture object and close all frames
    cap.release()
    cv2.destroyAllWindows()

track_qr_code()


In [9]:
import numpy as np

def select_roi():
    """Function to select an ROI from a randomly chosen frame in the video."""
    cap = cv2.VideoCapture(0)  # Start video capture
    
    while True:
        ret, frame = cap.read()
        if not ret:  
            break
        
        
    
        # Display the frame and let the user select ROI. Press ENTER or SPACE after selection.
        roi = cv2.selectROI("Select ROI", frame, False, False)
        
        
        break
            
    cv2.destroyAllWindows()
    cap.release()
    return frame, roi

def main():
    # Select ROI from a frame
    frame, roi = select_roi()
    roi_img = frame[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]

    # Initialize the ORB detector
    orb = cv2.ORB_create()

    # Compute keypoints and descriptors for the ROI
    kp_roi, des_roi = orb.detectAndCompute(roi_img, None)

    # Initialize the FLANN matcher
    index_params = dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=1)
    search_params = {}
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Start capturing video again for tracking
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect keypoints and compute descriptors in the current frame
        kp_frame, des_frame = orb.detectAndCompute(frame, None)

        # Match descriptors using FLANN matcher
         # Create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        
        # Match descriptors
        matches = bf.match(des_roi, des_frame)
        
        # Sort matches by distance (best first)
        matches = sorted(matches, key=lambda x: x.distance)

        # Apply Lowe's ratio test to filter matches
        good_matches = matches
        
        # Find homography to estimate the object's motion
        if len(good_matches) > 10:
            src_pts = np.float32([kp_roi[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

            if M is not None:
                h, w = roi_img.shape[:2]
                pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
                dst = cv2.perspectiveTransform(pts, M)
                frame = cv2.polylines(frame, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)

        cv2.imshow("Tracking", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
